In [1]:
from dotenv import load_dotenv
from langchain.tools import tool
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from pprint import pprint

load_dotenv()

True

## **Creating subagents**

In [2]:
@tool(description="Square a number")
def square(x: int) -> int:
    return x ** 2

@tool(description="Root of a number")
def root(x: int) -> int:
    return x ** (1/2)

In [3]:
sub_agent_1 = create_agent(
    model="gpt-5-nano",
    tools=[square],
)

sub_agent_2 = create_agent(
    model="gpt-5-nano",
    tools=[root],
)

In [4]:
@tool
def call_subagent_1(x: float) -> float:
    """Call subagent 1 in order to calculate the square root of a number"""
    response = sub_agent_1.invoke({"messages": [HumanMessage(content=f"Calculate the square root of {x}")]})
    return response["messages"][-1].content

@tool
def call_subagent_2(x: float) -> float:
    """Call subagent 2 in order to calculate the square of a number"""
    response = sub_agent_2.invoke({"messages": [HumanMessage(content=f"Calculate the square of {x}")]})
    return response["messages"][-1].content

In [5]:
## Creating the main agent

main_agent = create_agent(
    model='gpt-5-nano',
    tools=[call_subagent_1, call_subagent_2],
    system_prompt="You are a helpful assistant who can call subagents to calculate the square root or square of a number.")

## **Test**

In [6]:
question = "What is the square root of 456?"

response = main_agent.invoke({"messages": [HumanMessage(content=question)]})

In [7]:
pprint(response)

{'messages': [HumanMessage(content='What is the square root of 456?', additional_kwargs={}, response_metadata={}, id='bdc4c2c3-6b43-4b5e-9d34-dda2e448b90b'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 474, 'prompt_tokens': 202, 'total_tokens': 676, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 448, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-ConhKLUS5l9GYlwJ7jbOj8bwM7Cym', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b3b1f-6e99-75d3-b23f-7e34d8379013-0', tool_calls=[{'name': 'call_subagent_1', 'args': {'x': 456}, 'id': 'call_AjV2yb78JoIteQgSRvdkGY6p', 'type': 'tool_call'}], usage_metadata={'input_tokens': 202, 'output_tokens': 474, 'to